In [11]:
import pandas as pd
import os
import numpy as np

In [12]:
dogecoin = pd.read_csv("../data/doge_daily.tsv", sep='\t')

In [13]:
dogecoin.head()

,Date,Open,High,Low,Close*,Adj Close**,Volume
0,"Apr 15, 2021",0.1212,0.1873,0.1207,0.1822,0.1822,"17,916,620,386"
1,"Apr 14, 2021",0.0943,0.1425,0.0925,0.1215,0.1215,"20,977,679,327"
2,"Apr 13, 2021",0.0707,0.0952,0.0707,0.0934,0.0934,"7,176,815,089"
3,"Apr 12, 2021",0.0745,0.0750,0.0689,0.0708,0.0708,"2,687,720,380"
4,"Apr 11, 2021",0.0637,0.0792,0.0632,0.0746,0.0746,"6,646,627,317"


In [14]:
dogecoin['Date'] = pd.to_datetime(dogecoin['Date'])

In [19]:
dogecoin = dogecoin.set_index('Date').sort_index().reset_index()

In [20]:
dogecoin.head()

,Date,Open,High,Low,Close*,Adj Close**,Volume
0,2014-09-17,0.0003,0.0003,0.0003,0.0003,0.0003,"1,463,600"
1,2014-09-18,0.0003,0.0003,0.0003,0.0003,0.0003,"2,215,910"
2,2014-09-19,0.0003,0.0003,0.0003,0.0003,0.0003,"883,563"
3,2014-09-20,0.0003,0.0003,0.0003,0.0003,0.0003,"993,004"
4,2014-09-21,0.0003,0.0003,0.0003,0.0003,0.0003,"539,140"


In [29]:
dogecoin = dogecoin.rename(columns={'Close*' : 'Close'})

In [30]:
dogecoin['Daily_Change'] = dogecoin['Close'] - dogecoin['Open']
dogecoin['Daily_Change_Ind'] = np.where(dogecoin['Daily_Change'] > 0, 1, 0)

In [31]:
dogecoin.head()

,Date,Open,High,Low,Close,Adj Close**,Volume,Daily_Change,Daily_Change_Ind
0,"Apr 15, 2021",0.1212,0.1873,0.1207,0.1822,0.1822,"17,916,620,386",0.0610,1
1,"Apr 14, 2021",0.0943,0.1425,0.0925,0.1215,0.1215,"20,977,679,327",0.0272,1
2,"Apr 13, 2021",0.0707,0.0952,0.0707,0.0934,0.0934,"7,176,815,089",0.0227,1
3,"Apr 12, 2021",0.0745,0.0750,0.0689,0.0708,0.0708,"2,687,720,380",-0.0037,0
4,"Apr 11, 2021",0.0637,0.0792,0.0632,0.0746,0.0746,"6,646,627,317",0.0109,1


In [33]:
def expo_moving_average(df, period):
    '''
    Calculate the exponential moving average.
    
    Input:
        df: a pandas dataframe
        period (int): time period
    Returns:
        rv: a pd series of exponential moving averages
    '''
    col_name = f'EMA_{period}' 
    rv = pd.Series([None] * df.shape[0])
    weight_0 = 2 / (period + 1) 
    weight_1 = 1 - weight_0
    for row in df.itertuples():
        if row.Index < period - 1:
            continue
        elif row.Index == period - 1:
            simple_average = df.loc[:period - 1, 'Close'].sum() / period
            #df.loc[row.Index, col_name] = simple_average
            rv[row.Index] = simple_average
        else:
            #ema = row.Close * weight_0 + df.loc[row.Index - 1, col_name] * weight_1
            ema = row.Close * weight_0 + rv[row.Index - 1] * weight_1
            #df.loc[row.Index, col_name] = ema
            rv[row.Index] = ema
    return rv

In [34]:
def moving_average_convergence_divergence(df):
    '''
    Find the moving average convergence divergence, which is equal to the
    difference between the 12-period exponential moving average and
    26-period exponential moving average.
    
    Modifies the dataframe in place.
    
    Input:
        df: a pd dataframe
    '''
    col_name = 'MACD'
    ema_12 = expo_moving_average(dogecoin, 12)
    ema_26 = expo_moving_average(dogecoin, 26)
    df['MACD'] = ema_12 - ema_26

In [35]:
moving_average_convergence_divergence(dogecoin)

In [36]:
def proc(df, period):
    '''
    Calculate the price rate of change.
    
    Input:
        df: a pandas dataframe
        period (int): time period
    '''
    
    col_name = f'PROC_{period}'
    df[col_name] = None
    for row in df.itertuples():
        if row.Index < period:
            continue
        prev_idx = row.Index - period
        df.loc[row.Index, col_name] = (df.loc[row.Index, 'Close'] - df.loc[prev_idx, 'Close']) / df.loc[prev_idx, 'Close']

In [37]:
proc(dogecoin, 3)
proc(dogecoin, 5)
proc(dogecoin, 10)

In [40]:
def williams_percentage_range(df):
    '''
    Find the Williams Percentage Range. Modifies the df in place. 
    
    Input:
        df: a pd dataframe
    '''
    col_name = 'wpr'
    df[col_name] = None
    for row in df.itertuples():
        if row.Index < 14:
            continue
        max_14 = df.loc[row.Index - 14:row.Index - 1, 'Close'].max()
        min_14 = df.loc[row.Index - 14:row.Index - 1, 'Close'].min()
        try:
            wpr = (max_14 - row.Close) / (max_14 - min_14) * -100
        except:
            wpr = 0
        df.loc[row.Index, col_name] = wpr

In [41]:
williams_percentage_range(dogecoin)

In [45]:
def stochastic_oscillator(df):
    '''
    Find the stochastic oscillator. Modifies the df in place.
    '''
    col_name = 'sto_os'
    df[col_name] = None
    for row in df.itertuples():
        if row.Index < 14:
            continue
        max_14 = df.loc[row.Index - 14:row.Index - 1, 'Close'].max()
        min_14 = df.loc[row.Index - 14:row.Index - 1, 'Close'].min()
        try:
            so = (row.Close - min_14) / (max_14 - min_14) * 100
        except:
            so = 0
        df.loc[row.Index, col_name] = so

In [46]:
stochastic_oscillator(dogecoin)

In [47]:
dogecoin.head(30)

,Date,Open,High,Low,Close,Adj Close**,Volume,Daily_Change,Daily_Change_Ind,MACD,PROC_3,PROC_5,PROC_10,wpr,sto_os
0,"Apr 15, 2021",0.1212,0.1873,0.1207,0.1822,0.1822,"17,916,620,386",0.0610,1,NaN,None,None,None,None,None
1,"Apr 14, 2021",0.0943,0.1425,0.0925,0.1215,0.1215,"20,977,679,327",0.0272,1,NaN,None,None,None,None,None
2,"Apr 13, 2021",0.0707,0.0952,0.0707,0.0934,0.0934,"7,176,815,089",0.0227,1,NaN,None,None,None,None,None
3,"Apr 12, 2021",0.0745,0.0750,0.0689,0.0708,0.0708,"2,687,720,380",-0.0037,0,NaN,-0.611416,None,None,None,None
4,"Apr 11, 2021",0.0637,0.0792,0.0632,0.0746,0.0746,"6,646,627,317",0.0109,1,NaN,-0.386008,None,None,None,None
5,"Apr 10, 2021",0.0616,0.0651,0.0608,0.0638,0.0638,"1,369,793,117",0.0022,1,NaN,-0.316916,-0.649835,None,None,None
6,"Apr 09, 2021",0.0615,0.0639,0.0603,0.0617,0.0617,"1,060,465,773",0.0002,1,NaN,-0.128531,-0.492181,None,None,None
7,"Apr 08, 2021",0.0590,0.0617,0.0588,0.0615,0.0615,"1,055,257,541",0.0025,1,NaN,-0.175603,-0.341542,None,None,None
8,"Apr 07, 2021",0.0645,0.0673,0.0576,0.0590,0.0590,"2,857,060,848",-0.0055,0,NaN,-0.075235,-0.166667,None,None,None
9,"Apr 06, 2021",0.0597,0.0650,0.0581,0.0645,0.0645,"2,608,613,911",0.0048,1,NaN,0.045381,-0.135389,None,None,None


In [49]:
dogecoin.to_csv('../data/doge_daily_modified.csv')